In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn. preprocessing import OrdinalEncoder

In [5]:

df=pd.read_csv('covid_toy.csv')
df.head(3)

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No


In [6]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [7]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [9]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [ ]:
#for gender-- one hot encoding
#for fever-- simple imputer
#for cough-- ordinal encoder
#for city-- one hot encoding
#for label- no work yet

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [12]:
X_train

,age,gender,fever,cough,city
80,14,Female,99.0,Mild,Mumbai
69,73,Female,103.0,Mild,Delhi
6,14,Male,101.0,Strong,Bangalore
12,25,Female,99.0,Strong,Kolkata
34,74,Male,102.0,Mild,Mumbai
...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
5,84,Female,NaN,Mild,Bangalore
8,19,Female,100.0,Strong,Bangalore


<h2 style="color:#B00058">WITHOUT USING COLUMN TRANSFORMER</h2>

<h4 style="color:#FF0D86">APPLYING SIMPLE IMPUTER TO 'FEVER' COLUMN</h4>

In [14]:
si=SimpleImputer()
X_train_fever=si.fit_transform(X_train[['fever']])
X_test_fever=si.transform(X_test[['fever']])
X_test_fever

array([[ 98.        ],
       [ 98.        ],
       [101.        ],
       [100.81944444],
       [101.        ],
       [104.        ],
       [103.        ],
       [100.        ],
       [102.        ],
       [103.        ],
       [ 99.        ],
       [102.        ],
       [104.        ],
       [101.        ],
       [100.81944444],
       [ 98.        ],
       [100.        ],
       [101.        ],
       [102.        ],
       [100.        ]])

<h4 style="color:#FF0D86">APPLYING ORDINAL ENCODER TO 'COUGH' COLUMN</h4>

In [20]:
oe=OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough=oe.fit_transform(X_train[['cough']])

X_test_cough=oe.transform(X_test[['cough']])



<h4 style="color:#FF0D86">APPLYING ONEHOT ENCODING TO 'GENDER' AND 'CITY' COLUMNS</h4>

In [23]:
ohe=OneHotEncoder(drop='first',sparse_output=False)

X_train_gender_city=ohe.fit_transform(X_train[['gender','city']])

X_test_gender_city=ohe.transform(X_test[['gender','city']])


X_train_gender_city.shape
#one value removed from both the columns because of multi-linearity

(80, 4)

<h4 style="color:#FF0D86">EXTRACTING AGE</h4>

In [27]:
X_train_age=X_train.drop(columns=['gender','fever','cough','city']).values

X_test_age=X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

<h4 style="color:#FF0D86">MERGING ALL THE COLUMNS</h4>

In [30]:
X_train_transformed=np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)

X_test_transformed=np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

In [33]:
X_train_transformed.shape

(80, 7)

<h2 style="color:#B00058">USING COLUMN TRANSFORMER</h2>

In [44]:
from sklearn.compose import ColumnTransformer
transformer=ColumnTransformer(transformers=[
                  ('tnf1',SimpleImputer(),['fever']),
                  ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
                  ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [46]:
transformer.fit_transform(X_train).shape

(80, 7)

In [47]:
transformer.transform(X_test)

array([[ 98.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  24.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   5.        ],
       [101.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  51.        ],
       [100.81944444,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  34.        ],
       [101.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  64.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  51.        ],
       [103.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  60.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  10.        ],
       [102.        ,   1.        ,   0.        ,   0.        ,
          1.    